<a href="https://colab.research.google.com/github/mjcho1/practice_python/blob/master/rani_s_bookstore_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore", category = RuntimeWarning)
import io

#from google.colab import files
#uploaded = files.upload()

book = pd.read_csv('book_data.csv', encoding="ISO-8859-1")
print(book.shape)
book.head()

(54301, 12)


,book_authors,book_desc,book_edition,book_format,book_isbn,book_pages,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,Winning will make you famous. Losing means cer...,NaN,Hardcover,9780440000000,374 pages,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPrÃ©,There is a door at the end of a silent corrido...,US Edition,Paperback,9780440000000,870 pages,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,The unforgettable novel of a childhood in a sl...,50th Anniversary,Paperback,9780060000000,324 pages,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,Â«_ cosa ormai risaputa che a uno scapolo in p...,"Modern Library Classics, USA / CAN",Paperback,9780680000000,279 pages,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,About three things I was absolutely positive.F...,NaN,Paperback,9780320000000,498 pages,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...


In [ ]:
# 사용되는 컬럼 추출
book_df = book[['book_authors','book_isbn','book_rating','book_rating_count','book_review_count','book_title','genres','image_url']]
book_df.head()

,book_authors,book_isbn,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,9780440000000,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
1,J.K. Rowling|Mary GrandPrÃ©,9780440000000,4.48,2041594,33264,Harry Potter and the Order of the Phoenix,Fantasy|Young Adult|Fiction,https://images.gr-assets.com/books/1255614970l...
2,Harper Lee,9780060000000,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,9780680000000,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,9780320000000,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...


In [ ]:
# 중복되는 값 제거 (가상 ISBN)
book_df_2 = book_df.drop_duplicates(['book_isbn'], keep='first')
book_df_2.head()

,book_authors,book_isbn,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,9780440000000,4.33,5519135,160706,The Hunger Games,Young Adult|Fiction|Science Fiction|Dystopia|F...,https://images.gr-assets.com/books/1447303603l...
2,Harper Lee,9780060000000,4.27,3745197,79450,To Kill a Mockingbird,Classics|Fiction|Historical|Historical Fiction...,https://images.gr-assets.com/books/1361975680l...
3,Jane Austen|Anna Quindlen|Mrs. Oliphant|George...,9780680000000,4.25,2453620,54322,Pride and Prejudice,Classics|Fiction|Romance,https://images.gr-assets.com/books/1320399351l...
4,Stephenie Meyer,9780320000000,3.58,4281268,97991,Twilight,Young Adult|Fantasy|Romance|Paranormal|Vampire...,https://images.gr-assets.com/books/1361039443l...
5,Markus Zusak,9780380000000,4.36,1485632,100821,The Book Thief,Historical|Historical Fiction|Fiction|Young Adult,https://images.gr-assets.com/books/1522157426l...


In [ ]:
#book_df_2['book_isbn'] object -> int64 로 타입 변경
book_df_2['book_isbn'] = pd.to_numeric(book_df_2['book_isbn'],errors='coerce')
book_df_2['book_isbn']=book_df_2['book_isbn'].astype('Int64')
book_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 549 entries, 0 to 53157
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   book_authors       549 non-null    object 
 1   book_isbn          539 non-null    Int64  
 2   book_rating        549 non-null    float64
 3   book_rating_count  549 non-null    int64  
 4   book_review_count  549 non-null    int64  
 5   book_title         549 non-null    object 
 6   genres             513 non-null    object 
 7   image_url          530 non-null    object 
dtypes: Int64(1), float64(1), int64(2), object(4)
memory usage: 39.1+ KB


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [ ]:
# 결측치 제거
book_df_2.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
# book_df_2['genres'] 의 '|' 제거
book_df_2['genres'] = book_df_2['genres'].astype('str').apply(lambda x : x.split('|'))
# book_df_2['genres_literal'] : CountVectorizer 를 적용하기 위해 공백 문자로 word 단위가 구분되는 문자열로 변환
book_df_2['genres_literal'] = book_df_2['genres'].apply(lambda x : (' ').join(x))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 문자열로 변환된 genres 칼럼을 Count 기반으로 피처 벡터화 변환
count_vect = CountVectorizer(min_df=0, ngram_range=(1,2))
# fit_transform 함수 적용
genre_mat = count_vect.fit_transform(book_df_2['genres_literal'])
# cosine_similarity 함수 적용 : 코사인 유사도 계산, 기준 행과 비교 행의 코사인 유사도를 행렬 행태로 반환 하는 함수
genre_sim = cosine_similarity(genre_mat, genre_mat)
print(genre_sim.shape)
print(genre_sim[:5])

(500, 500)
[[1.         0.28005602 0.25819889 ... 0.         0.21821789 0.        ]
 [0.28005602 1.         0.43386092 ... 0.         0.1833397  0.        ]
 [0.25819889 0.43386092 1.         ... 0.         0.16903085 0.        ]
 [0.33596842 0.10585122 0.19518001 ... 0.         0.08247861 0.        ]
 [0.4472136  0.62622429 0.23094011 ... 0.         0.19518001 0.        ]]


In [ ]:
# 0번 레코드의 비교 행 위치 인덱스 값만 샘플로 추출
genre_sim_sort = genre_sim.argsort()[:, :: -1]
# test code print 로 코사인 유사도 확인
print(genre_sim[0][0], genre_sim[0][5])
# test code print 로 책 제목 확인
print(book_df_2['book_title'][5], book_df_2['book_title'][0])

1.0000000000000002 0.4061811972299616
The Book Thief The Hunger Games


In [ ]:
# 임의의 isbn_data 입력 (The Hunger Games)
isbn_data = 9780440000000

# find_sim_book 함수
def find_sim_book(df, sorted_ind, isbn, top_n=10):
    print("find_sim_book")
    isbn_book = df[df['book_isbn'] == isbn]
    isbn_index = isbn_book.index.values
    similar_indexes = sorted_ind[isbn_index, :(top_n)]
    # dataframe에서 index로 사용하기 위해서 1차원 array로 변경
    similar_indexes = similar_indexes.reshape(-1)
    # similar_indexes 값 return
    return df.iloc[similar_indexes]

# find_sim_book 함수 적용 : 코사인 유사도
similar_books = find_sim_book(book_df_2, genre_sim_sort, isbn_data, 10)

# sort_values() 를 이용해 'book_rating' 기준 오름차순으로 정렬해서 10개 출력
similar_books[['book_authors','book_isbn','book_rating','book_rating_count','book_review_count','book_title',
    'genres','image_url']].sort_values('book_rating', ascending=False)[:10]

find_sim_book


,book_authors,book_isbn,book_rating,book_rating_count,book_review_count,book_title,genres,image_url
0,Suzanne Collins,9780440000000,4.33,5519135,160706,The Hunger Games,"[Young Adult, Fiction, Science Fiction, Dystop...",https://images.gr-assets.com/books/1447303603l...
25,Orson Scott Card,9780810000000,4.30,965351,40289,Ender's Game,"[Science Fiction, Fiction, Young Adult, Fantasy]",https://images.gr-assets.com/books/1408303130l...
23440,Stephen King,9782280000000,4.29,136,10,Le FlÃ©au 1/3,[Science Fiction],https://images.gr-assets.com/books/1388319686l...
27211,Ð¡Ð²Ð¾Ð»Ð°Ð² Ð¾Ð³Ð¸Ð½Ð¾Ð²,9785870000000,4.19,194,14,Ð½Ð¾Ð³Ð¾ÐºÐ¸Ð¹ Ð±Ð¾Ð³ Ð´Ð°Ð»Ð°Ð¹Ð½Ð°,"[Fantasy, Cultural, Russia, Fiction, Science F...",https://images.gr-assets.com/books/1319046079l...
11757,Roger Zelazny,9789990000000,4.18,18605,291,The Courts of Chaos,"[Fantasy, Fiction, Science Fiction]",https://images.gr-assets.com/books/1417786728l...
13992,George Orwell|Alexandre Hubner|HeloÃ­sa Jahn,9788540000000,4.16,2422432,53809,1984,"[Classics, Fiction, Science Fiction, Science F...",https://images.gr-assets.com/books/1293505059l...
19302,Margaret Atwood,9785810000000,4.09,946348,51168,The Handmaid's Tale (Contemporary Classics),"[Fiction, Classics, Science Fiction, Dystopia,...",https://images.gr-assets.com/books/1500064636l...
241,Scott Westerfeld,9780690000000,3.86,503415,22315,Uglies,"[Young Adult, Science Fiction, Dystopia, Scien...",https://images.gr-assets.com/books/1443904172l...
11961,Stephenie Meyer,9782710000000,3.84,812594,40588,Les _mes vagabondes,"[Science Fiction, Fiction, Young Adult, Scienc...",https://images.gr-assets.com/books/1260225161l...
2239,G.D. Falksen,9781430000000,3.63,216,36,Blood in the Skies,"[Science Fiction, Steampunk, Fantasy, Science ...",https://images.gr-assets.com/books/1310488542l...


In [ ]:
# 가중 평점 적용
C = book_df_2['book_rating'].mean()
m = book_df_2['book_review_count'].quantile(0.6)
    
def weighted_vote_average(record):
    v = record['book_review_count']
    R = record['book_rating']
    return ( (v/(v+m)) * R ) + ( (m/(m+v)) * C )

book_df_2['weighted_vote'] = book_df_2.apply(weighted_vote_average, axis=1)
book_df_2[['book_authors','book_isbn','book_rating','book_rating_count','book_review_count','book_title',
    'genres','image_url','weighted_vote']].sort_values('weighted_vote', ascending=False)[:10]
    

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


,book_authors,book_isbn,book_rating,book_rating_count,book_review_count,book_title,genres,image_url,weighted_vote
183,Cassandra Clare,9781410000000,4.59,277242,25357,Clockwork Princess,"[Fantasy, Young Adult, Romance, Fantasy, Paran...",https://images.gr-assets.com/books/1436788488l...,4.573578
21854,Sarah J. Maas|Marcin Mortka,9788330000000,4.59,155945,17153,KrÃ³lowa cieni,"[Fantasy, Young Adult, Romance]",https://images.gr-assets.com/books/1460469210l...,4.566102
18652,Patrick Rothfuss|Vera Ribeiro,9788600000000,4.55,547196,33632,O Nome do Vento,"[Fantasy, Fiction, Fantasy, Epic Fantasy]",https://images.gr-assets.com/books/1427160065l...,4.538522
21475,Patrick Rothfuss|Jana Re_kovÃ¡,9788030000000,4.55,547203,33632,JmÃ©no v_tru,"[Fantasy, Fiction, Fantasy, Epic Fantasy]",https://images.gr-assets.com/books/1357244476l...,4.538522
2906,Patrick Rothfuss|Gemma Rovira,9788400000000,4.55,547009,33625,El nombre del viento,"[Fantasy, Fiction, Fantasy, Epic Fantasy]",https://images.gr-assets.com/books/1419003669l...,4.538520
120,Patrick Rothfuss,9780760000000,4.55,546719,33615,The Name of the Wind,"[Fantasy, Fiction, Fantasy, Epic Fantasy]",https://images.gr-assets.com/books/1515589515l...,4.538517
28,Kathryn Stockett,9780400000000,4.46,1790905,80741,The Help,"[Fiction, Historical, Historical Fiction, Hist...",https://images.gr-assets.com/books/1346100365l...,4.456104
428,Cassandra Clare,9781480000000,4.47,234021,21056,City of Heavenly Fire,"[Fantasy, Young Adult, Fantasy, Paranormal, Ro...",https://images.gr-assets.com/books/1460477794l...,4.455100
623,Leigh Bardugo,9781630000000,4.46,165056,24638,Six of Crows,"[Fantasy, Young Adult]",https://images.gr-assets.com/books/1459349344l...,4.447533
44,George R.R. Martin,9780550000000,4.45,1657040,49398,A Game of Thrones,"[Fantasy, Fiction, Fantasy, Epic Fantasy]",https://images.gr-assets.com/books/1436732693l...,4.443846


In [ ]:
def find_sim_book_improve(df, sorted_ind, isbn, top_n=10):
    # 함수 호출 확인하는 print
    print("find_sim_book_improve")
    isbn_book = df[df['book_isbn'] == isbn]
    isbn_index = isbn_book.index.values
    # sorted_ind 인자로 입력된 genre_sim_sorted_ind 객체에서 유사도 순으로 top_n*2개의 index 추출
    similar_indexes = sorted_ind[isbn_index, :(top_n*2)]
    similar_indexes = similar_indexes.reshape(-1)
    # 기존 영화 index는 제외
    similar_indexes = similar_indexes[similar_indexes != isbn_index]
    # top_n의 2배에 해당하는 후보군에서 weighted_vote 높은 순으로 top_n 만큼 추출
    return df.iloc[similar_indexes].sort_values('weighted_vote', ascending=False)[:3]

similar_books2 = find_sim_book_improve(book_df_2, genre_sim_sort, isbn_data, 10)

# 최종 상위 3개 추천 결과 출력
similar_books2.head()

find_sim_book_improve


,book_authors,book_isbn,book_rating,book_rating_count,book_review_count,book_title,genres,image_url,genres_literal,weighted_vote
25,Orson Scott Card,9780810000000,4.30,965351,40289,Ender's Game,"[Science Fiction, Fiction, Young Adult, Fantasy]",https://images.gr-assets.com/books/1408303130l...,Science Fiction Fiction Young Adult Fantasy,4.295643
464,Stephen King,9781450000000,4.30,344956,33174,11/22/1963,"[Fiction, Historical, Historical Fiction, Scie...",https://images.gr-assets.com/books/1327876792l...,Fiction Historical Historical Fiction Science ...,4.294732
13992,George Orwell|Alexandre Hubner|HeloÃ­sa Jahn,9788540000000,4.16,2422432,53809,1984,"[Classics, Fiction, Science Fiction, Science F...",https://images.gr-assets.com/books/1293505059l...,Classics Fiction Science Fiction Science Ficti...,4.158939
